In [ ]:
import math
import numpy as np

In [ ]:
# Node class representing nodes in the decision tree
class Node:
  def __init__(self, node_type, attr, value):
    self.node_type = node_type
    self.attr = attr
    self.value = value
    self.children = []

  def append_child(self, child):
    self.children.append(child)

In [ ]:
# Entropy function to calculate the entropy between the 2 given values
def entropy_2vals(e, p):
  if e == 0 or p == 0: return 0
  sum = float(e+p)
  return -1*(e/sum*math.log(e/sum, 2)) - (p/sum*math.log(p/sum, 2))

In [ ]:
# Gets entropies for given working set
def entropy(working_set):
  # Calculate all entropy values
  entropies = {}
  for col in working_set.columns:
    if col == 'points': continue

    unique_attr = working_set[col].unique()
    freq_table = pd.DataFrame(data=np.zeros((2,len(unique_attr)), dtype=np.int64), index=[0,1], columns=unique_attr)

    for row in working_set[col].index:
      attr = working_set[col][row]
      freq_table[attr][df_train['points'][row]] += 1

    entro = 0
    if len(working_set.index) > 0:
      for freq_col in freq_table.columns:
        entro += (freq_table[freq_col].sum()/len(working_set.index)) * entropy_2vals(freq_table[freq_col][0], freq_table[freq_col][1])

    entropies[col] = entro

  return entropies

In [ ]:
# Gets entropy of e column
def class_entropy(rows):
  # Get working set of rows
  working_set = df_train.iloc[rows, [0]]

  # Calculate entropy for classes
  value_counts = working_set.value_counts().values
  if len(value_counts) < 2:
    return 0
  else:
    return entropy_2vals(working_set.value_counts().values[0], working_set.value_counts().values[1])

In [ ]:
# Recursive function to make the decision tree
def make_tree(learning_set):
  # Get class entropy given attribute values
  class_entro = class_entropy(learning_set.index)

  # Get entropies
  entropies = entropy(learning_set)

  # Check if homogenous group or not and respond appropriately
  if class_entro == 0:
    leaf_node = Node(node_type='leaf', attr=None, value=df_train['points'][learning_set.index[0]])
    return leaf_node
  else:
    # Calculate the information gains
    info_gains = {}
    for entro in entropies:
      info_gains[entro] = class_entro - entropies[entro]

    if len(learning_set.columns) == len(df_train.columns) - 1:
      print("Information gains for iteration 1:")
      print(info_gains)

    root_col = max(info_gains, key=info_gains.get)
    root = Node(node_type='decision', attr=root_col, value=None)

    for attr in learning_set[root_col].unique():
      attr_node = Node(node_type='attr', attr=root_col, value=attr)
      new_columns = np.delete(learning_set.columns.values, np.where(learning_set.columns.values == root_col))
      new_learning_set = learning_set[learning_set[root_col]==attr]
      new_learning_set = new_learning_set.loc[:, new_columns]
      attr_node.append_child(make_tree(new_learning_set))
      root.append_child(attr_node)

    return root

In [ ]:
# Returns the predicted value for the given rowe in the test_data_frame
def traverse_tree(decision_tree, row):
  if decision_tree.node_type == 'leaf':
    return decision_tree.value
  elif decision_tree.node_type == 'attr':
    return traverse_tree(decision_tree.children[0], row)
  else:
    for child in decision_tree.children:
      if child.value == df_test[child.attr][row]:
        return traverse_tree(child, row)
        break;

In [ ]:
# Prints the tree (for testing purposes)
def print_tree(the_tree):
  print('(', end='')
  print(the_tree.node_type, end='')
  print(str(the_tree.attr), end='')
  print(str(the_tree.value), end='')
  for child in the_tree.children:
    print_tree(child)
  print(')', end='')

In [ ]:
# Get the raw data
train_data_url = 'https://raw.githubusercontent.com/ptsheeran/AI/main/TrainingData.csv'
test_data_url = 'https://raw.githubusercontent.com/ptsheeran/AI/main/TestingData.csv'

# Get a data frame for the train data
df_train = pd.read_csv(train_data_url)
df_test = pd.read_csv(test_data_url)

In [ ]:
# Replace unknown values
for col in df_train.columns:
  for row in df_train[col].index:
    if df_train[col][row] == '?':
      mode = df_train[col].mode().values[0]
      df_train[col][row] = mode

for col in df_test.columns:
  for row in df_test[col].index:
    if df_test[col][row] == '?':
      mode = df_test[col].mode().values[0]
      df_test[col][row] = mode

In [ ]:
# Start with all the attributes, except decision column
starting_attributes = df_train.columns[:len(df_train.columns)-2]

# Make the starting learning set, which is everything except the first column
starting_learning_set = df_train.loc[:,starting_attributes]

# Make the decision tree
tree = make_tree(starting_learning_set)

# Testing the tree
total_correct = 0
for row in df_test.index:
  if traverse_tree(tree, row) == df_test['points'][row]:
    total_correct += 1

print("Decision Tree Accuracy: " + str((total_correct/len(df_test.index))*100) + "%")

Decision Tree Accuracy: 82.83378746594006%


In [ ]:
def Perceptron(X, Y, learningRate=0.01, maxIter=100):
    np.random.seed(1)
    N = X.shape[1]
    D_plus_1 = X.shape[0]
    w = np.zeros((D_plus_1, 1))
    w += .1

    for t in range(maxIter):
        permutation = np.random.permutation(N)
        X = X[:, permutation]
        Y = Y[permutation, :] 
        for n in range(N):
            y1 = np.sign(np.dot(np.transpose(w), X[:, n]))
            if (y1 == -1):
              y1 += 1
            if (y1 != Y[n, :]):
              if (Y[n, :] == 0):
                w += learningRate * -1 * X[:, n].reshape(-1,1)
              else:
                w += learningRate * np.transpose((np.dot(Y[n, :].reshape(-1,1), np.transpose(X[:, n].reshape(-1,1)))))

    return w

In [ ]:
def Accuracy(X, Y, w):
  Y_hat = np.sign(np.matmul(X.transpose(), w))
  Y_hat = np.where(Y_hat == -1, 0, Y_hat)
  correct = (Y_hat == Y)
  return math.fsum(correct) / len(correct)

In [ ]:
#Start of logistic regression
import pandas as pd
log_train_data_url = 'https://raw.githubusercontent.com/ptsheeran/AI/main/LogTrainingData.csv'
log_test_data_url = 'https://raw.githubusercontent.com/ptsheeran/AI/main/LogTestingData.csv'
#Get the data CSV files we created from github for simplicity
log_df_train = pd.read_csv(log_train_data_url)
log_df_test = pd.read_csv(log_test_data_url)

In [ ]:
import pandas as pd
#find the mean
testValues = 0
trainValues = 0
avgTest = []
avgTrain = []
sum = 0.0
for col in log_df_test.columns:
  for row in log_df_test[col].index:
    if log_df_test[col][row] != '?':
      #print(row, sum,testValues,log_df_test[col][row])
      sum += float(log_df_test[col][row])
      testValues +=1
      
    
  sum = sum / testValues 
  avgTest.append(sum)
  sum = 0.0
  testValues = 0

sum = 0.0
#find the mean of the train columns
for col in log_df_train.columns:
  for row in log_df_train[col].index:
    if log_df_train[col][row] != '?':
      sum += float(log_df_train[col][row])
      trainValues +=1
  sum = sum / trainValues 
  avgTrain.append(sum)
  sum = 0.0
  trainValues = 0

# Replace unknown values
colNum = 0
for col in log_df_train.columns:
  for row in log_df_train[col].index:
    if log_df_train[col][row] == '?':
      log_df_train[col][row] = avgTrain[colNum]
  colNum +=1

colNum = 0
for col in log_df_test.columns:
  for row in log_df_test[col].index:
    if log_df_test[col][row] == '?':
      log_df_test[col][row] = avgTest[colNum]
  colNum +=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
# Start with all the attributes, except decision column
working_attributes = log_df_train.columns[:len(log_df_train.columns)-2]

X_train = np.asarray(log_df_train.loc[:, working_attributes].T, dtype=np.float64)
Y_train = np.asarray(log_df_train['points'], dtype=np.float64).reshape(len(log_df_train.index), 1)

X_test = np.asarray(log_df_test.loc[:, working_attributes].T, dtype=np.float64)
Y_test = np.asarray(log_df_test['points'], dtype=np.float64).reshape(len(log_df_test.index), 1)

w = Perceptron(X_train, Y_train,  maxIter=100, learningRate=.1)

training_accuracy = Accuracy(X_train, Y_train, w)
test_accuracy = Accuracy(X_test, Y_test, w)
print("Perceptron Accuracy: training set: ", training_accuracy)
print("Perceptron Accuracy: test set: ", test_accuracy)

Perceptron Accuracy: training set:  0.8977444747199516
Perceptron Accuracy: test set:  0.9046321525885559
